In [20]:
#deal with tensors
import numpy as np
import sklearn.metrics
import torch
import spacy
from torch.utils.data import random_split
from torchtext.legacy import data
import torchtext.vocab
import random
import helper.sklearn_to_csv
from sklearn import metrics

In [21]:
#Cuda algorithms
#torch.backends.cudnn.deterministic = True

In [22]:
COL_OPINION = 1
COL_SPEECH = 9
TOTAL_COLS = 16

RUNS = 100
BATCH_SIZE = 16
TEST_SPLIT = 0.15
VALID_SPLIT = 0.15
SHUFFLE = True
STRATIFY = True
CLASS_WEIGHTS = True
REMOVE_STOP_WORDS = False
STEMMING = False
LOWERING = True
NO_PUNCTUATION = False

ROOT = '../..'
OUT_NAME = f'measures_lstm_01_{TEST_SPLIT}_{VALID_SPLIT}_Shuff{SHUFFLE}_Strat{STRATIFY}_Weigh{CLASS_WEIGHTS}_NoStop{REMOVE_STOP_WORDS}_Stem{STEMMING}_Low{LOWERING}_NoPun{NO_PUNCTUATION}'
PATH_METRICS = f'{ROOT}/results/{OUT_NAME}.csv'
PATH_DATASET = f"{ROOT}/protocols/labelled/massnahmen_relevant.csv"
PATH_PERFORMANCE_GRAPH = f'{ROOT}/results/{OUT_NAME}.png'
PATH_WEIGHTS = f'stored_models/{OUT_NAME}.pt'

if not LOWERING:
    raise Exception("Lower-casing must be enabled!")

In [23]:
# COL_OPINION = 6
# COL_SPEECH = 13
# TOTAL_COLS = 14
#
# RUNS = 1
# BATCH_SIZE = 16
# TEST_SPLIT = 0.15
# VALID_SPLIT = 0.0
# SHUFFLE = True
# STRATIFY = True
# CLASS_WEIGHTS = True
# REMOVE_STOP_WORDS = False
# STEMMING = False
# LOWERING = True
# NO_PUNCTUATION = False
#
# ROOT = '../..'
# OUT_NAME = f'lockdown_lstm_01_{TEST_SPLIT}_{VALID_SPLIT}_Shuff{SHUFFLE}_Strat{STRATIFY}_Weigh{CLASS_WEIGHTS}_NoStop{REMOVE_STOP_WORDS}_Stem{STEMMING}_Low{LOWERING}_NoPun{NO_PUNCTUATION}'
# PATH_METRICS = f'{ROOT}/results/{OUT_NAME}.csv'
# PATH_DATASET = f"{ROOT}/protocols/labelled/only_lockdown.csv"
# PATH_PERFORMANCE_GRAPH = f'{ROOT}/results/{OUT_NAME}.png'
# PATH_WEIGHTS = f'stored_models/{OUT_NAME}.pt'
#
# if not LOWERING:
#     raise Exception("Lower-casing must be enabled!")

In [24]:
nlp = spacy.load("de_dep_news_trf")

In [25]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)
fields = [(None, None)] * TOTAL_COLS
fields[COL_OPINION] = ('label', LABEL)
fields[COL_SPEECH] = ('text',TEXT)



In [26]:
# loading custom dataset
training_data=data.TabularDataset(path = PATH_DATASET,format = 'csv',fields = fields,skip_header = True,
                                  csv_reader_params={'delimiter': '\t', 'quotechar': None})

In [27]:
all_samples = [e for e in training_data.examples]

In [28]:
# Pre-processing
# to lower case to match with vector embeddings
tmp = []
for sample in all_samples:
    text = getattr(sample, 'text')
    text = [s.lower() for s in text]
    setattr(sample, 'text', text)

In [29]:
#initialize glove embeddings
# the custom vectors we use in de_glove_deepsetai.txt have 300 dimensions
de_embeddings = torchtext.vocab.Vectors(name = f'{ROOT}/vectors/de_glove_deepsetai.txt')
TEXT.build_vocab(training_data,min_freq=1,vectors = de_embeddings)
LABEL.build_vocab(training_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))

#Word dictionary
print(TEXT.vocab.stoi)

Size of TEXT vocabulary: 18050
Size of LABEL vocabulary: 3
[(',', 22767), ('.', 9099), ('die', 8465), ('und', 7066), ('der', 5533), ('das', 4537), ('in', 3919), ('es', 3665), ('ist', 3489), ('wir', 3335)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f7883f03f40>>, {'<unk>': 0, '<pad>': 1, ',': 2, '.': 3, 'die': 4, 'und': 5, 'der': 6, 'das': 7, 'in': 8, 'es': 9, 'ist': 10, 'wir': 11, '–': 12, 'sie': 13, 'nicht': 14, 'dass': 15, 'auch': 16, 'den': 17, 'zu': 18, 'ich': 19, ':': 20, 'haben': 21, 'mit': 22, 'i': 23, 'm': 24, 'für': 25, 'von': 26, '-': 27, 'sind': 28, 'eine': 29, 'hat': 30, '\xa0': 31, 'ein': 32, 'sich': 33, 'werden': 34, 'man': 35, 'auf': 36, 'wie': 37, 'wird': 38, 'aber': 39, 'jetzt': 40, 'wenn': 41, 'bei': 42, '!': 43, 'noch': 44, 'da': 45, 'dann': 46, 'so': 47, 'diese': 48, 'des': 49, 'dem': 50, 'an': 51, 'lockdown': 52, 'was': 53, 'um': 54, 'ja': 55, 'als': 56, 'einen': 57, 'können': 58, 'gibt': 59, 'nur': 60, 'menschen': 61,

In [30]:
LABEL.vocab.stoi['-'] = 0
LABEL.vocab.stoi['o'] = 1
LABEL.vocab.stoi['+'] = 2
LABEL.vocab.itos = ['-', 'o', '+']

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
import torch.nn as nn

class classifier(nn.Module):

    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout):

        #Constructor
        super().__init__()

        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #lstm layer
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout,
                           batch_first=True)

        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        #activation function
        self.act = nn.Sigmoid()

    def forward(self, text, text_lengths):

        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]

        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]

        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)

        return outputs

In [33]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 300  # the custom vectors we use in de_glove_deepsetai.txt have 300 dimensions
num_hidden_nodes = 32
num_output_nodes = 3  # we have 3 different labels (-/o/+) so we need 3 output nodes.
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers,
                   bidirectional = True, dropout = dropout)


In [34]:
import torch.optim as optim


#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.argmax(preds, dim=-1)

    correct = (rounded_preds == y)
    acc = correct.sum() / len(correct)
    return acc



In [35]:
def train(model, iterator, optimizer, criterion):

    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #set the model in training phase
    model.train()

    for batch in iterator:

        #resets the gradients after every batch
        optimizer.zero_grad()

        #retrieve text and no. of words
        text, text_lengths = batch.text

        # predictions has list of likelihoods for the 3 classes for each sample.
        predictions = model(text, text_lengths)

        #compute the loss
        loss = criterion(predictions, batch.label.type(torch.LongTensor))

        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label.type(torch.LongTensor))

        #backpropage the loss and compute the gradients
        loss.backward()

        #update the weights
        optimizer.step()

        #loss and accuracy
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    actual_labels = []
    predicted_labels = []
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths)
            actual_labels += batch.label.type(torch.LongTensor).tolist()
            predicted_labels += np.argmax(predictions, axis=-1).tolist()
            #compute loss and accuracy
            loss = criterion(predictions, batch.label.type(torch.LongTensor))
            acc = binary_accuracy(predictions, batch.label.type(torch.LongTensor))
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), metrics.classification_report(actual_labels, predicted_labels, output_dict=True)

In [ ]:
def random_train_test_split(ds, split=1.0-TEST_SPLIT):
    positive = [e for e in ds if getattr(e, 'label') == '+']
    negative = [e for e in ds if getattr(e, 'label') == '-']
    neutral = [e for e in ds if getattr(e, 'label') == 'o']

    neg_train = int(len(negative) * split)
    neut_train = int(len(neutral) * split)
    pos_train = int(len(positive) * split)

    if SHUFFLE:
        random.shuffle(positive)
        random.shuffle(negative)
        random.shuffle(neutral)
    train_data = data.Dataset(positive[:pos_train] + negative[:neg_train] + neutral[:neut_train], fields)
    valid_data = data.Dataset(positive[pos_train:] + negative[neg_train:] + neutral[neut_train:], fields)

    return train_data, valid_data, neg_train, neut_train, pos_train

def create_iterators(train_data, test_data):
    train_iterator, valid_iterator = data.BucketIterator.splits(
        (train_data, test_data),
        batch_size = BATCH_SIZE,
        sort_key = lambda x: len(x.text),
        sort_within_batch=True,
        shuffle=True,
        device = device)
    return train_iterator, valid_iterator


N_EPOCHS = 10
runs = RUNS
accuracies = []

for run in range(runs):
    if VALID_SPLIT > 0:
        train_data, test_data, _, _, _ = random_train_test_split(all_samples, 1.0-TEST_SPLIT)
        split_train_, split_valid_, neg_train, neut_train, pos_train = random_train_test_split(train_data, 1.0-VALID_SPLIT)
    else:
        train_data, test_data, neg_train, neut_train, pos_train = random_train_test_split(all_samples, 1.0-TEST_SPLIT)
        split_valid_ = train_data
        split_train_ = train_data

    train_iterator, valid_iterator = create_iterators(split_train_, split_valid_)
    test_iterator = data.BucketIterator(test_data, BATCH_SIZE, train=False, sort_key = lambda x: len(x.text), device = device)

    model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers,
                   bidirectional = True, dropout = dropout)
    #Initialize the pretrained embedding
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    #define optimizer and loss
    optimizer = optim.Adam(model.parameters())

    if CLASS_WEIGHTS:
        max_count = max([neg_train, neut_train, pos_train])
        criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([max_count/float(neg_train), max_count/float(neut_train), max_count/float(pos_train)]))
    else:
        criterion = nn.CrossEntropyLoss()

    #push to cuda if available
    model = model.to(device)
    criterion = criterion.to(device)

    best_valid_loss = float('inf')
    best_valid_acc = 0.0
    for epoch in range(N_EPOCHS):
        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, classification_report = evaluate(model, valid_iterator, criterion)

        #save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_valid_acc = valid_acc
            torch.save(model.state_dict(), PATH_WEIGHTS)

        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # load best model
    model.load_state_dict(torch.load(PATH_WEIGHTS))
    model.eval()
    test_loss, test_acc, classification_report = evaluate(model, test_iterator, criterion)
    helper.sklearn_to_csv.dict_to_csv(classification_report, PATH_METRICS)
    accuracies.append(test_acc)

	Train Loss: 1.059 | Train Acc: 54.21%
	 Val. Loss: 0.992 |  Val. Acc: 54.98%
	Train Loss: 0.919 | Train Acc: 64.99%
	 Val. Loss: 0.928 |  Val. Acc: 62.30%
	Train Loss: 0.799 | Train Acc: 77.73%
	 Val. Loss: 0.910 |  Val. Acc: 62.60%
	Train Loss: 0.690 | Train Acc: 87.55%
	 Val. Loss: 0.885 |  Val. Acc: 64.48%
	Train Loss: 0.625 | Train Acc: 93.21%
	 Val. Loss: 0.876 |  Val. Acc: 65.35%
	Train Loss: 0.596 | Train Acc: 96.19%
	 Val. Loss: 0.896 |  Val. Acc: 64.23%
	Train Loss: 0.590 | Train Acc: 96.48%
	 Val. Loss: 0.896 |  Val. Acc: 65.04%
	Train Loss: 0.585 | Train Acc: 96.86%
	 Val. Loss: 0.881 |  Val. Acc: 68.80%
	Train Loss: 0.569 | Train Acc: 98.35%
	 Val. Loss: 0.898 |  Val. Acc: 68.19%
	Train Loss: 0.566 | Train Acc: 98.65%
	 Val. Loss: 0.882 |  Val. Acc: 67.07%


/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

	Train Loss: 1.085 | Train Acc: 43.31%
	 Val. Loss: 1.024 |  Val. Acc: 57.27%
	Train Loss: 0.953 | Train Acc: 62.14%
	 Val. Loss: 0.932 |  Val. Acc: 64.84%
	Train Loss: 0.858 | Train Acc: 71.91%
	 Val. Loss: 0.901 |  Val. Acc: 68.60%
	Train Loss: 0.785 | Train Acc: 78.28%
	 Val. Loss: 0.872 |  Val. Acc: 68.80%
	Train Loss: 0.708 | Train Acc: 83.89%
	 Val. Loss: 0.863 |  Val. Acc: 69.05%
	Train Loss: 0.674 | Train Acc: 89.05%
	 Val. Loss: 0.851 |  Val. Acc: 69.21%
	Train Loss: 0.612 | Train Acc: 94.55%
	 Val. Loss: 0.852 |  Val. Acc: 70.17%
	Train Loss: 0.590 | Train Acc: 96.47%
	 Val. Loss: 0.858 |  Val. Acc: 69.36%
	Train Loss: 0.580 | Train Acc: 97.51%
	 Val. Loss: 0.863 |  Val. Acc: 71.49%
	Train Loss: 0.574 | Train Acc: 97.89%
	 Val. Loss: 0.846 |  Val. Acc: 72.36%
	Train Loss: 1.068 | Train Acc: 51.30%
	 Val. Loss: 0.978 |  Val. Acc: 62.96%
	Train Loss: 0.933 | Train Acc: 64.03%
	 Val. Loss: 0.942 |  Val. Acc: 62.04%
	Train Loss: 0.820 | Train Acc: 75.42%
	 Val. Loss: 0.915 |  Val

/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

	Train Loss: 1.068 | Train Acc: 47.53%
	 Val. Loss: 1.032 |  Val. Acc: 58.74%
	Train Loss: 0.964 | Train Acc: 60.98%
	 Val. Loss: 0.957 |  Val. Acc: 63.16%
	Train Loss: 0.897 | Train Acc: 66.24%
	 Val. Loss: 0.954 |  Val. Acc: 64.13%
	Train Loss: 0.875 | Train Acc: 68.68%
	 Val. Loss: 0.978 |  Val. Acc: 57.62%
	Train Loss: 0.829 | Train Acc: 76.02%
	 Val. Loss: 0.956 |  Val. Acc: 58.94%
	Train Loss: 0.703 | Train Acc: 87.41%
	 Val. Loss: 0.871 |  Val. Acc: 67.73%
	Train Loss: 0.629 | Train Acc: 93.29%
	 Val. Loss: 0.874 |  Val. Acc: 69.56%
	Train Loss: 0.597 | Train Acc: 95.81%
	 Val. Loss: 0.857 |  Val. Acc: 69.61%
	Train Loss: 0.586 | Train Acc: 96.75%
	 Val. Loss: 0.871 |  Val. Acc: 68.19%
	Train Loss: 0.583 | Train Acc: 97.04%
	 Val. Loss: 0.880 |  Val. Acc: 69.87%


...

	Train Loss: 1.061 | Train Acc: 51.06%
	 Val. Loss: 0.998 |  Val. Acc: 59.71%
	Train Loss: 0.932 | Train Acc: 63.35%
	 Val. Loss: 0.944 |  Val. Acc: 64.68%
	Train Loss: 0.845 | Train Acc: 72.25%
	 Val. Loss: 0.959 |  Val. Acc: 65.65%
	Train Loss: 0.802 | Train Acc: 77.34%
	 Val. Loss: 0.919 |  Val. Acc: 67.94%
	Train Loss: 0.719 | Train Acc: 87.30%
	 Val. Loss: 0.933 |  Val. Acc: 68.29%
	Train Loss: 0.626 | Train Acc: 94.01%
	 Val. Loss: 0.919 |  Val. Acc: 66.87%
	Train Loss: 0.591 | Train Acc: 96.70%
	 Val. Loss: 0.868 |  Val. Acc: 66.72%
	Train Loss: 0.577 | Train Acc: 97.65%
	 Val. Loss: 0.891 |  Val. Acc: 71.49%
	Train Loss: 0.572 | Train Acc: 98.13%
	 Val. Loss: 0.867 |  Val. Acc: 72.36%
	Train Loss: 0.568 | Train Acc: 98.48%
	 Val. Loss: 0.869 |  Val. Acc: 69.61%
	Train Loss: 1.065 | Train Acc: 49.95%
	 Val. Loss: 0.974 |  Val. Acc: 59.30%
	Train Loss: 0.946 | Train Acc: 63.28%
	 Val. Loss: 0.924 |  Val. Acc: 64.99%
	Train Loss: 0.855 | Train Acc: 74.01%
	 Val. Loss: 0.902 |  Val

/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

	Train Loss: 1.067 | Train Acc: 50.80%
	 Val. Loss: 0.996 |  Val. Acc: 59.86%
	Train Loss: 0.936 | Train Acc: 64.29%
	 Val. Loss: 0.949 |  Val. Acc: 61.48%
	Train Loss: 0.811 | Train Acc: 77.18%
	 Val. Loss: 1.007 |  Val. Acc: 64.18%
	Train Loss: 0.759 | Train Acc: 81.40%
	 Val. Loss: 0.892 |  Val. Acc: 66.92%
	Train Loss: 0.671 | Train Acc: 89.00%
	 Val. Loss: 0.885 |  Val. Acc: 67.28%
	Train Loss: 0.618 | Train Acc: 94.29%
	 Val. Loss: 0.869 |  Val. Acc: 68.39%
	Train Loss: 0.595 | Train Acc: 95.85%
	 Val. Loss: 0.877 |  Val. Acc: 66.57%
	Train Loss: 0.596 | Train Acc: 95.67%
	 Val. Loss: 0.895 |  Val. Acc: 67.94%
	Train Loss: 0.590 | Train Acc: 96.40%
	 Val. Loss: 0.895 |  Val. Acc: 68.50%
	Train Loss: 0.579 | Train Acc: 97.19%
	 Val. Loss: 0.891 |  Val. Acc: 68.95%
	Train Loss: 1.067 | Train Acc: 52.77%
	 Val. Loss: 0.978 |  Val. Acc: 63.01%
	Train Loss: 0.914 | Train Acc: 65.74%
	 Val. Loss: 0.916 |  Val. Acc: 66.26%
	Train Loss: 0.808 | Train Acc: 77.42%
	 Val. Loss: 0.953 |  Val

/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

	Train Loss: 1.070 | Train Acc: 49.91%
	 Val. Loss: 1.004 |  Val. Acc: 61.84%
	Train Loss: 0.933 | Train Acc: 64.53%
	 Val. Loss: 0.948 |  Val. Acc: 63.01%
	Train Loss: 0.798 | Train Acc: 78.32%
	 Val. Loss: 0.902 |  Val. Acc: 63.72%
	Train Loss: 0.685 | Train Acc: 88.44%
	 Val. Loss: 0.908 |  Val. Acc: 67.38%
	Train Loss: 0.623 | Train Acc: 93.56%
	 Val. Loss: 0.906 |  Val. Acc: 68.45%
	Train Loss: 0.599 | Train Acc: 95.64%
	 Val. Loss: 0.877 |  Val. Acc: 70.58%
	Train Loss: 0.585 | Train Acc: 96.86%
	 Val. Loss: 0.892 |  Val. Acc: 66.62%
	Train Loss: 0.577 | Train Acc: 97.59%
	 Val. Loss: 0.880 |  Val. Acc: 66.72%
	Train Loss: 0.575 | Train Acc: 98.05%
	 Val. Loss: 0.890 |  Val. Acc: 66.72%


In [ ]:
# plot accuracies
import matplotlib.pyplot as plt
plt.plot(sorted(accuracies), label='Model accuracies')
plt.axhline(y=0.33, color='r', linestyle='-', label='Random guess')
plt.axhline(y=sum(accuracies)/runs, color='orange', linestyle='-', label='Mean accuracy')
plt.ylabel('Model Accuracies')
plt.title(f"Accuracies of {runs} individual train-evaluation runs")
plt.legend(loc="lower right", borderaxespad=0)
# plt.savefig(PATH_PERFORMANCE_GRAPH)
plt.show()


In [ ]:
def predict_sentence(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence
    return predict(model, tokenized)


def predict(model, tokenized):
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction
    return prediction

tens = predict_sentence(model, "Ich bin ein Lockdown.")


In [ ]:
#make predictions
sentence = "Ich bin ein Lockdown."
probs = predict_sentence(model, sentence)
label = LABEL.vocab.itos[torch.argmax(probs)]
print(sentence + " => " + label)

In [ ]:
# determine validation accuracy
correct = 0
total = 0
for e in valid_data:
    # print(getattr(e,'label'))
    # print(getattr(e, 'text'))
    target_label = getattr(e, 'label')
    tokenized_sentence = getattr(e, 'text')
    preds = predict(model, tokenized_sentence)
    pred_label = LABEL.vocab.itos[torch.argmax(preds)]
    if pred_label == target_label:
        correct += 1
    total += 1

accuracy = correct / total

In [ ]:

print(model)